## Nucleus Segmentation with a pre-trained network

lorem ipsum https://bioimage.io/#/

In [ ]:
import os

import h5py
import napari

In [ ]:
data_folder = "../data"
output_folder = os.path.join("../data", "predictions")
os.makedirs(output_folder, exist_ok=True)

### 1. Load the nucleus segmentation model

https://bioimage.io/#/?tags=nuclei&id=10.5281%2Fzenodo.5764892

In [ ]:
import bioimageio.core
from xarray import DataArray

In [ ]:
model_name = "affable-shark"

model = bioimageio.core.load_resource_description(model_name)

### 2. Check the model

In [ ]:
image_path = "../data/test/gt_image_040.h5"
with h5py.File(image_path, "r") as f:
    image = f["raw/nuclei/s0"][:]

In [ ]:
with bioimageio.core.create_prediction_pipeline(model) as pp:
    input_ = DataArray(image[None, None], dims=tuple("bcyx"))
    prediction = pp(input_)[0].squeeze().values

In [ ]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_image(prediction)

### 3. Implement post-processing to get an instance segmentation

In [ ]:
import numpy as np
from scipy.ndimage import distance_transform_edt
from skimage.feature import peak_local_max
from skimage.filters import gaussian
from skimage.segmentation import watershed

In [ ]:
foreground, boundaries = prediction
foreground = foreground > 0.5
boundary_distances = distance_transform_edt(boundaries < 0.1)
boundary_distances[~foreground] = 0
boundary_distances = gaussian(boundary_distances)

In [ ]:
seed_points = peak_local_max(boundary_distances, min_distance=5, exclude_border=False)
seeds = np.zeros(foreground.shape, dtype="uint32")
seeds[seed_points[:, 0], seed_points[:, 1]] = np.arange(1, len(seed_points) + 1)
nucleus_segmentation = watershed(boundaries, markers=seeds, mask=foreground)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_image(boundaries)
viewer.add_image(boundary_distances)
viewer.add_points(seed_points)
viewer.add_labels(nucleus_segmentation)

### 4. Apply to all data

In [ ]:
from glob import glob
from tqdm import tqdm

In [ ]:
input_files = glob(os.path.join(data_folder, "*.h5"))

In [ ]:
shapes = [np.array(h5py.File(path, "r")["raw/nuclei/s0"].shape) for path in input_files]
print("Image shapes:", np.unique(shapes, axis=0))

In [ ]:
print(model.inputs[0].shape)

In [ ]:
def run_segmentation(pp, image, min_distance=5, sigma=1.0):
    input_ = DataArray(image[None, None], dims=tuple("bcyx"))
    prediction = bioimageio.core.predict_with_padding(pp, input_, padding={"x": 16, "y": 16})[0].squeeze().values
    foreground, boundaries = prediction
    foreground = foreground > 0.5
    boundary_distances = distance_transform_edt(boundaries < 0.1)
    boundary_distances[~foreground] = 0
    boundary_distances = gaussian(boundary_distances, sigma)
    seed_points = peak_local_max(boundary_distances, min_distance=min_distance, exclude_border=False)
    seeds = np.zeros(foreground.shape, dtype="uint32")
    seeds[seed_points[:, 0], seed_points[:, 1]] = np.arange(1, len(seed_points) + 1)
    nucleus_segmentation = watershed(boundaries, markers=seeds, mask=foreground)
    return nucleus_segmentation

In [ ]:
with bioimageio.core.create_prediction_pipeline(model) as pp:
    for path in tqdm(input_files, desc="Run nucleus segmentation"):
        with h5py.File(path, "r") as f:
            image = f["raw/nuclei/s0"][:]
        segmentation = run_segmentation(pp, image)
        output_path = os.path.join(output_folder, os.path.basename(path))
        with h5py.File(output_path, "a") as f:
            f.create_dataset("segmentations/nuclei/watershed_based", data=segmentation, compression="gzip")

### Exercises

lorem ipsum dolor

**What's next**